In [1]:
import requests
import random
import pandas as pd
import re
import json
import datetime as dt
from bs4 import BeautifulSoup
from headers import headers_list
from loading_bar import log_progress
from data_skills import SKILLS, SKILL_DICT
from skill_extraction import extract_skills, extract_ignore
from secrets import api_keys

In [2]:
URL = 'https://www.googleapis.com/youtube/v3/search'
ORDERS = ['date', 'rating', 'relevance', 'title', 'videoCount', 'viewCount']

API_KEY = api_keys['youtube']
# Must be in RFC 3339 formatted date-time value (1970-01-01T00:00:00Z)
START_DATE = (dt.datetime.now(dt.timezone.utc) - dt.timedelta(days=365.25)).isoformat()
START_DATE

'2020-09-28T20:03:20.159583+00:00'

In [3]:
def get_youtube_videos(query, max_results=50, order='viewCount', start_date=None):
    params = {'q': query, 'part': 'snippet', 'type': 'video', 'maxResults': max_results, 'order': order,
              'key': API_KEY}
    if start_date is not None:
        params['publishedAfter'] = start_date
    page = requests.get(URL, params=params, headers=random.choice(headers_list))
    if page.status_code != 200:
        return None
    res = json.loads(page.content.decode('utf8'))
    video_dict = []
    video_url = 'https://youtube.com/watch?v='
    for i, item in enumerate(res['items']):
        video_id = get_object(item['id'], 'videoId')
        snippet = item['snippet']
        video_dict.append({
            'id': video_id,
            'title': get_object(snippet, 'title'),
            'channel': get_object(snippet, 'channelTitle'),
            'url': video_url + video_id,
            'published_date': get_object(snippet, 'publishTime'),
            'description': get_object(snippet, 'description'),
            'relevance': i+1
        })
    return pd.DataFrame.from_dict(video_dict)

def get_object(item, key):
    try:
        return item[key]
    except:
        return None

In [4]:
df = pd.DataFrame()

for i in log_progress(range(len(SKILLS))):
    skill = SKILLS[i]
    query = 'learn ' + skill
    df_temp = get_youtube_videos(query, order='relevance', start_date=START_DATE)
    df_temp['data_skills'] = skill
    df = df.append(df_temp)

df.head()

,id,title,channel,url,published_date,description,relevance,data_skills
0,S2MUhGA3lEw,Spark Full Course | Spark Tutorial For Beginne...,Simplilearn,https://youtube.com/watch?v=S2MUhGA3lEw,2021-05-18T18:54:13Z,This Apache Spark full course will help you le...,1,APACHE
1,UeRkNU2zMUA,"Apache numbers, greetings and history",East Fork Lutheran School,https://youtube.com/watch?v=UeRkNU2zMUA,2020-10-02T15:55:54Z,,2,APACHE
2,QLQsW8VbTN4,Apache Spark / PySpark Tutorial: Basics In 15 ...,Greg Hogg,https://youtube.com/watch?v=QLQsW8VbTN4,2021-03-25T22:42:20Z,Apache Spark / PySpark Tutorial in 15 minutes!...,3,APACHE
3,aReuLtY0YMI,Hadoop In 5 Minutes | What Is Hadoop? | Introd...,Simplilearn,https://youtube.com/watch?v=aReuLtY0YMI,2021-01-21T13:30:05Z,Free Big Data Hadoop and Spark Developer cours...,4,APACHE
4,2v9AKewyUEo,Learning Apache Airflow with Python in easy wa...,soumilshah1995,https://youtube.com/watch?v=2v9AKewyUEo,2021-01-24T15:05:38Z,Code https://github.com/soumilshah1995/Learn-A...,5,APACHE


In [5]:
len(df), len(df.drop_duplicates(subset=['id']))

(1850, 1797)

In [6]:
df = df.merge(df.groupby('id').mean(), left_on='id', right_index=True)
df = df.merge(df[['id', 'data_skills']].groupby('id').agg(lambda x: '; '.join(x)), left_on='id', right_index=True)
df.head()

,id,title,channel,url,published_date,description,relevance_x,data_skills_x,relevance_y,data_skills_y
0,S2MUhGA3lEw,Spark Full Course | Spark Tutorial For Beginne...,Simplilearn,https://youtube.com/watch?v=S2MUhGA3lEw,2021-05-18T18:54:13Z,This Apache Spark full course will help you le...,1,APACHE,9.666667,APACHE; SPARK; Hadoop
0,S2MUhGA3lEw,Spark Full Course | Spark Tutorial For Beginne...,Simplilearn,https://youtube.com/watch?v=S2MUhGA3lEw,2021-05-18T18:54:13Z,This Apache Spark full course will help you le...,1,SPARK,9.666667,APACHE; SPARK; Hadoop
26,S2MUhGA3lEw,Spark Full Course | Spark Tutorial For Beginne...,Simplilearn,https://youtube.com/watch?v=S2MUhGA3lEw,2021-05-18T18:54:13Z,This Apache Spark full course will help you le...,27,Hadoop,9.666667,APACHE; SPARK; Hadoop
1,UeRkNU2zMUA,"Apache numbers, greetings and history",East Fork Lutheran School,https://youtube.com/watch?v=UeRkNU2zMUA,2020-10-02T15:55:54Z,,2,APACHE,2.000000,APACHE
2,QLQsW8VbTN4,Apache Spark / PySpark Tutorial: Basics In 15 ...,Greg Hogg,https://youtube.com/watch?v=QLQsW8VbTN4,2021-03-25T22:42:20Z,Apache Spark / PySpark Tutorial in 15 minutes!...,3,APACHE,3.000000,APACHE; SPARK


In [7]:
df = df.drop_duplicates(subset=['id'])
df = df.drop(columns=['relevance_x', 'data_skills_x'])
df = df.rename(columns={'relevance_y': 'relevance', 'data_skills_y': 'data_skills'})
df = df.reset_index(drop=True)
df.head()

,id,title,channel,url,published_date,description,relevance,data_skills
0,S2MUhGA3lEw,Spark Full Course | Spark Tutorial For Beginne...,Simplilearn,https://youtube.com/watch?v=S2MUhGA3lEw,2021-05-18T18:54:13Z,This Apache Spark full course will help you le...,9.666667,APACHE; SPARK; Hadoop
1,UeRkNU2zMUA,"Apache numbers, greetings and history",East Fork Lutheran School,https://youtube.com/watch?v=UeRkNU2zMUA,2020-10-02T15:55:54Z,,2.000000,APACHE
2,QLQsW8VbTN4,Apache Spark / PySpark Tutorial: Basics In 15 ...,Greg Hogg,https://youtube.com/watch?v=QLQsW8VbTN4,2021-03-25T22:42:20Z,Apache Spark / PySpark Tutorial in 15 minutes!...,3.000000,APACHE; SPARK
3,aReuLtY0YMI,Hadoop In 5 Minutes | What Is Hadoop? | Introd...,Simplilearn,https://youtube.com/watch?v=aReuLtY0YMI,2021-01-21T13:30:05Z,Free Big Data Hadoop and Spark Developer cours...,2.500000,APACHE; Hadoop
4,2v9AKewyUEo,Learning Apache Airflow with Python in easy wa...,soumilshah1995,https://youtube.com/watch?v=2v9AKewyUEo,2021-01-24T15:05:38Z,Code https://github.com/soumilshah1995/Learn-A...,5.000000,APACHE


In [44]:
def get_video_info(video_id):
    url = 'https://www.googleapis.com/youtube/v3/videos'
    params = {'part': ['snippet', 'statistics'], 'id': video_id, 'key': API_KEY}
    page = requests.get(url, params=params, headers=random.choice(headers_list))
    if page.status_code != 200:
        return None
    try:
        res = json.loads(page.content.decode('utf8'))
        return res
    except:
        return None

def get_snippet(res, info):
    try:
        return res['items'][0]['snippet'][info]
    except:
        return None

def get_statistics(res, info):
    try:
        return res['items'][0]['statistics'][info]
    except:
        return None

In [46]:
for i in log_progress(range(0, len(df))):
    row = df.loc[i]
    full_text = row['title']
    video_info = get_video_info(row['id'])
    if video_info is None:
        continue
    full_description = get_snippet(video_info, 'description')
    df.loc[i, 'description_full'] = full_description
    df.loc[i, 'language'] = get_snippet(video_info, 'defaultAudioLanguage')
    df.loc[i, 'view_count'] = get_statistics(video_info, 'viewCount')
    df.loc[i, 'like_count'] = get_statistics(video_info, 'likeCount')
    df.loc[i, 'dislike_count'] = get_statistics(video_info, 'dislikeCount')
    df.loc[i, 'comment_count'] = get_statistics(video_info, 'commentCount')
    # Get skills from description
    if full_description is not None:
        full_text += ' ' + full_description
    else:
        full_text += ' ' + row['description']
    all_skills = extract_skills(full_text)
    keep_skills, _ = extract_ignore(all_skills)
    keep_skills.sort()
    if len(keep_skills) > 0:
        df.loc[i, 'skills'] = '; '.join(keep_skills)

df.head()

,id,title,channel,url,published_date,description,relevance,data_skills,description_full,skills,language,view_count,like_count,dislike_count,comment_count
371,--NQfX4qoNA,Learn Microsoft Excel Basics Tutorial for Begi...,Excel Dude Microsoft Excel Tutorials for Begin...,https://youtube.com/watch?v=--NQfX4qoNA,2020-12-28T20:16:33Z,Microsoft Excel 365 tutorial for beginners lea...,24.0,Excel,Microsoft Excel 365 tutorial for beginners lea...,Formatting; Golang; Microsoft Excel,en-US,2723,78,2,15
1617,-4jokJpvFwc,Learn JavaScript — A Tutorial For Beginners (2...,CareerFoundry,https://youtube.com/watch?v=-4jokJpvFwc,2021-01-08T17:22:25Z,Learn the basics of JavaScript! Build your ver...,20.0,JavaScript,Learn the basics of JavaScript! Build your ver...,Blogging; Google Chrome; JavaScript; Links; So...,None,3978,53,5,12
353,-5fCvAtl8b8,Excel Full Course | Microsoft Excel Course | M...,Simplilearn,https://youtube.com/watch?v=-5fCvAtl8b8,2021-05-04T15:46:57Z,This Excel Tutorial full course will help you ...,6.0,Excel,This Excel Tutorial full course will help you ...,Analytical; Android; Big Data; Business Analys...,en,37320,1475,9,125
155,-7Q41t0qO7M,MongoDB Tutorial | MongoDB Tutorial For Beginn...,Simplilearn,https://youtube.com/watch?v=-7Q41t0qO7M,2021-05-29T15:30:00Z,"""This video on MongoDB tutorial will help you ...",6.0,MongoDB,"""This video on MongoDB tutorial will help you ...",Analytical; Big Data; Configuration; Data Mode...,en,3322,120,3,18
425,-F9-NTX1qp0,Learn Tableau With Tableau Public 12: Bubble V...,Amit Chandak,https://youtube.com/watch?v=-F9-NTX1qp0,2021-08-25T17:57:25Z,Learn Tableau With Tableau Public 12: Bubble V...,28.0,Tableau,Learn Tableau With Tableau Public 12: Bubble V...,Github; Microsoft Power BI; Tableau; Video,en,86,1,0,0


In [47]:
df.loc[df['skills'].isna()]

,id,title,channel,url,published_date,description,relevance,data_skills,description_full,skills,language,view_count,like_count,dislike_count,comment_count
1200,IyzuiJAzjzk,PSQF7375_Longitudinal_Lecture2_Part1,Lesa Hoffman,https://youtube.com/watch?v=IyzuiJAzjzk,2021-02-02T23:11:55Z,Lecture 2 slides 1-31.,40.0,Regressions,Lecture 2 slides 1-31,None,en,145,4,0,5


In [48]:
df = df[['id', 'title', 'channel', 'url', 'published_date', 'description', 'language', 'view_count', 'like_count',
         'dislike_count', 'comment_count', 'relevance', 'description_full', 'data_skills', 'skills']]
df = df.sort_values(by='id')
df.drop(columns=['description_full']).to_csv('results/youtube_2021.csv', index=False)
df.to_csv('results/youtube_full_2021.csv', index=False)